# Pistachio Irrigation Water Optimization

This Jupyter Notebook contains Python code that optimizes the irrigation water for pistachio cultivation. The code reads plant information and irrigation data from an Excel file, performs various calculations, and then optimizes the irrigation schedule to maximize expected crop yield while minimizing water consumption.

## Required Packages

We use following packages to optimize irrigation:

a. pandas: Pandas is a popular data manipulation library in Python. It provides data structures like DataFrame and Series, and functions for reading, writing, and manipulating data efficiently.

b. math.prod: The prod() function from the math module calculates the product of all elements in an iterable.

c. tabulate: The tabulate package is used to display the final results in a tabulated format for better readability.

d. scipy.optimize.minimize: This function is used for numerical optimization. It attempts to find the minimum of a scalar function by iteratively adjusting the input parameters.

In [39]:
import pandas as pd
from math import prod
from tabulate import tabulate
from scipy.optimize import minimize

## Function Descriptions

The code consists of several functions that perform specific tasks. Below are the descriptions of each function:

### `read_input(cellName)`

This function reads the value associated with a given cell name from the 'Input' sheet in the Excel file. It takes the cell name as input and returns the corresponding value or None if the cell is not found.


In [40]:
inputSheet = pd.read_excel('data.xlsx', sheet_name='Input', skiprows=1, nrows=5, usecols="A:B")
print(inputSheet)
def read_input(cellName):
    global inputSheet
    df = inputSheet
    df.columns = ['key', 'value']
    key_row = df[df['key'] == cellName]
    
    if not key_row.empty:
        value = key_row.iloc[0]['value']
        return value
    else:
        return None


    Soil Water Solidity       5
0  Irrigation Frequency    40.0
1   Expected Crop Yeild  5000.0
2        Field Capacity   157.5
3                   Age    10.0


### `get_expected_productivity()`

This function calculates the expected productivity of the pistachio plant based on input parameters such as expected crop yield, soil water solidity, and plant age. It adjusts the expected crop yield based on the soil water solidity and plant age, and then returns the expected productivity.


In [41]:
def get_expected_productivity():
    expected = read_input('Expected Crop Yeild')
    soilWaterSolidity = read_input('Soil Water Solidity')
    age = read_input('Age')
    if soilWaterSolidity and soilWaterSolidity > 7:
        expected = 100 - 3.6 * (expected - 7)
    if(age):
        if(age < 3):
            expected *= .4
        elif age < 6:
            expected *= .7
        elif age < 9:
            expected *= .9
    return expected


### `get_max_productivity()`

This function extracts the maximum crop efficiency from the 'Crop History' sheet in the Excel file. It returns the maximum crop efficiency value.


In [42]:
cropHistory = pd.read_excel('data.xlsx', sheet_name='Crop History', skiprows=1, nrows=9, usecols="B:B")

def get_max_productivity():
    global cropHistory
    df = cropHistory
    df.columns = ['Efficiency']
    maxCropEfficiency = df['Efficiency'].max()
    return maxCropEfficiency


### `create_dataframe()`

This function reads the 'Irrigation' sheet from the Excel file and creates a DataFrame with the required columns for irrigation data.


In [43]:
def create_dataframe():
    df = pd.read_excel('data.xlsx', sheet_name='Irrigation', skiprows=1, nrows=23, usecols="A:H")
    pd.set_option('expand_frame_repr', False)
    df.columns = ['Month', 'Decade', 'Progress Stage', 'Kc', 'ETo', 'ETc', 'Effective Precipitation', 'Irrigation']
    return df
df = create_dataframe()
print(df)

          Month  Decade  Progress Stage    Kc    ETo    ETc  Effective Precipitation  Irrigation
0     Farvardin       2            0.50  0.38  42.93  12.85                     4.61        8.24
1     Farvardin       3            0.50  0.38  55.07  16.48                     2.89       13.59
2   Ordibehesht       1            0.85  0.43  53.85  18.23                     0.00       18.23
3   Ordibehesht       2            0.85  0.49  53.46  20.63                     1.89       18.74
4   Ordibehesht       3            0.85  0.54  64.76  27.54                     0.57       26.97
5       Khordad       1            0.85  0.60  64.18  30.33                     0.00       30.33
6       Khordad       2            0.85  0.58  67.12  30.66                     0.00       30.66
7       Khordad       3            0.85  0.58  77.65  35.47                     0.00       35.47
8           Tir       1            0.85  0.58  72.93  33.31                     0.00       33.31
9           Tir       2       

### `calculate_columns(df)`

This function performs various calculations to create new columns in the DataFrame. It calculates Raes Method1, Raes Method2, Raes Method3, Raes Method4, and irrigation scheduling for each month based on the irrigation frequency.


In [44]:
def calculate_columns(df):
    df['Raes Method1'] = df['Progress Stage'] * (df['Irrigation Scheduling'] / df['Irrigation'])
    df['Raes Method2'] = 1 - df['Raes Method1']
    df['Raes Method3'] = pow(df['Raes Method2'], 10/220)
    df['Raes Method4'] = 1 - df['Raes Method3']
    cycle = read_input('Irrigation Frequency')
    cycle_key = f'Irrigation Scheduling F={cycle}'
    df[cycle_key] = None
    jump = cycle / 10
    for i in range(0, len(df)):
        if (i % jump) == 0:
            sum = df.loc[i : i + jump - 1, 'Irrigation Scheduling'].sum()
            capacity = read_input('Field Capacity')
            df.loc[i, cycle_key] = sum if capacity > sum else capacity 


          Month  Decade  Progress Stage    Kc    ETo    ETc  Effective Precipitation  Irrigation  Irrigation Scheduling  Raes Method1  Raes Method2  Raes Method3  Raes Method4 Irrigation Scheduling F=40.0
0     Farvardin       2            0.50  0.38  42.93  12.85                     4.61        8.24                   8.24          0.50          0.50      0.968984      0.031016                         58.8
1     Farvardin       3            0.50  0.38  55.07  16.48                     2.89       13.59                  13.59          0.50          0.50      0.968984      0.031016                         None
2   Ordibehesht       1            0.85  0.43  53.85  18.23                     0.00       18.23                  18.23          0.85          0.15      0.917381      0.082619                         None
3   Ordibehesht       2            0.85  0.49  53.46  20.63                     1.89       18.74                  18.74          0.85          0.15      0.917381      0.082619     

### `calculate_objective(df)`

This function calculates the objective value for optimization. It uses the product of Raes Method3, the maximum crop efficiency, and the expected productivity to determine the objective value.


In [46]:
def calculate_objective(df):
    product = df['Raes Method3'].product() 
    max = get_max_productivity()
    expected = get_expected_productivity()
    objective = expected - ((1 - product) * max)
    return objective


### `objective_function(x, df)`

This function defines the objective function for optimization. It updates the irrigation scheduling in the DataFrame based on the input parameters x, calculates the objective value using calculate_objective(), and returns the absolute value of the objective.


In [47]:
def objective_function(x, df):
    df['Irrigation Scheduling'] = x
    calculate_columns(df)
    objective = calculate_objective(df)
    return abs(objective) 


### `constraint(x, x_values, y_values)`

This function defines the constraint for optimization. It ensures that the optimized irrigation scheduling is less than or equal to the original irrigation values.


In [48]:
def constraint(x, x_values, y_values):
    return y_values - x_values


In [49]:
# Perform optimization
result = minimize(objective_function, df['Irrigation Scheduling'].values, args=(df,), constraints={'type': 'ineq', 'fun': constraint, 'args': (df['Irrigation'].values, df['Irrigation Scheduling'].values), }, bounds=[(0, None)])
optimized_y_values = result.x
df['Irrigation Scheduling'] = optimized_y_values
calculate_columns(df)
optimized_objective = calculate_objective(df)

# Display results
print("Optimized Objective Value:", optimized_objective)
print("Total Irrigation Water: ", df['Irrigation Scheduling'].sum())

table = tabulate(df, headers=df.columns, tablefmt='outline')
print(table)

Optimized Objective Value: 2.7939677238464355e-09
Total Irrigation Water:  430.1229769879437
+----+-------------+----------+------------------+------+-------+-------+---------------------------+--------------+-------------------------+----------------+----------------+----------------+----------------+--------------------------------+
|    | Month       |   Decade |   Progress Stage |   Kc |   ETo |   ETc |   Effective Precipitation |   Irrigation |   Irrigation Scheduling |   Raes Method1 |   Raes Method2 |   Raes Method3 |   Raes Method4 |   Irrigation Scheduling F=40.0 |
+====+=============+==========+==================+======+=======+=======+===========================+==============+=========================+================+================+================+================+================================+
|  0 | Farvardin   |        2 |             0.5  | 0.38 | 42.93 | 12.85 |                      4.61 |         8.24 |                 7.23254 |       0.438868 |       0.561132 